# IMEC2201 Herramientas Computacionales 
## Taller Semana 6: Optimización
### Fecha Límite Entrega (Bloque Neón): <font color="#FF5733">23.59h de Mayo 22, 2022</font>

Universidad de los Andes — Mayo 11, 2022.
___
### Tener en Cuenta
La entrega del Taller 6 se puede realizar de tres maneras:
1. Cargar la solución en su repositorio en GitHub y enviar el enlace del mismo vía Bloque Neón.
2. Enviar vía Bloque Neón el archivo en Jupyter Notebook (extensión '.ipynb').
3. Enviar vía Bloque Neón el archivo PDF del Jupyter Notebook.
___

# Semana 6: Optimizacion

En esta semana se trabajaran diferentes casos de optimización, desde bombas hasta turbinas eólicas. Este tipo de ejercicios se realizan en ingeniería mecánica para el diseño de diferentes sistemas de bombeo o de conversión de energía. En complemento a lo visto en clase, se dejara un taller práctico con el fin de optimizar diferentes parámetros de bombas y turbinas eólicas. 

In [8]:
# Importar paquetes
using JuMP
using GLPK
using Ipopt
using Plots
using Dierckx
using DataFrames
using Interpolations
using Roots 
using NLsolve

LoadError: ArgumentError: Package JuMP [4076af6c-e467-56ae-b986-b466b2749572] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


# Bomba Centrifuga

Se necesita conocer la cabeza máxima de una bomba ficticia, cuya eficiencia es constante en un 30%. Opera entre 1000 y 2000 RPM, y el diámetro mínimo es de 10 centímetros. 

**Nota:** Buscar las propiedades hidráulicas del agua a temperatura ambiente.

#### Detalles
Recuerden que la eficiencia de la bomba es:

$$
\eta = \frac{P_w}{P_f} = \frac{\rho g Q H}{w T}
$$

Donde la potencia que la bomba centrífuga le añade al fluido (conocida como **potencia hidráulica** $P_w$), es:

$$
P_w = \rho g Q H
$$

Y para que la bomba centrífuga le añada energía al fluido, debe haber un recurso que, asimismo, le suministre energía al dispositivo. Esto último es conocido como **potencia mecánica** $P_f$ y es el producto entre la velocidad de rotación del eje de la bomba $w$ y el torque en el mismo $T$.

$$
P_f = w T
$$

También:

$$
Q = VA
$$

y

$$
A = \frac{\pi D^2}{4}
$$

In [7]:
function bomba_centrifuga()
    
    ρ = 1.225 
    g = 9.81
    n = 0.30
    V = 1
    H = 1 
    # Paso 1
    model = Model(GLPK.Optimizer)
    # Paso 2
    @variable(model, 2000 >= w >= 1000)
    @variable(model, d > 0.10)
    # Paso 3
    @objective(model, Max, (ρ*g *V* H * π/4 * d^2)/(w*T))
    # Paso 4
    @constraint(model, c1,(ρ*g *V* H * π/4 * d^2)/(w*T) = 0.30) #esto restringe el la velocidad angular de la turbina_eolica
    # Paso 5
    optimize!(model)
    # Paso 6
    @show objective_value(model)
    return @show value(d)
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[7]:11

# Turbina Eólica

Para un parque eólico en La Guajira, se tomaron medidas de velocidad del viento, las cuales, segun el sitio https://www.windy.com/ oscilan entre 15 y 22 nudos. Si se planea que la velocidad angular de la turbina no supere las 3000 RPM, encontrar el mínimo diámetro posible con el fin de extraer 60 GW de potencia del viento. 

**Nota:** Tener en cuenta que la potencia eólica es: $P_w = \frac{1}{2}\rho A v^3$ donde $\rho$ es la densidad del aire. 

#### Detalles
Recuerde que:
- 1 nudo equivale a 0.51 m/s.
- La densidad $\rho$ del aire es 1.225 kg/m$^3$.

También, el área $A$ es:

$$
A = \frac{\pi D^2}{4}
$$

In [5]:
function turbina_eolica()
    ρ = 1.225 
    n = 0.51
    # Paso 1
    model = Model(GLPK.Optimizer)
    # Paso 2
    @variable(model, 22 >= v >= 15)
    # Paso 3
    @objective(model, Min, 0.5*ρ*(π/4)*d^2*v^3)
    # Paso 4
    @constraint(model, c1, 2*v*n/d < 3000) #esto restringe el la velocidad angular de la turbina_eolica
    @constraint(model, c2, 0.5*ρ*(π/4)*d^2*v^3 = 60*10^9)
    # Paso 5
    optimize!(model)
    # Paso 6
    @show objective_value(model)
    return @show value(d)# De nuevo, escribir la respuesta final aquí
end
turbina_eolica()

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[5]:7